## LinkedIn Profile Visit and Connection Request

In [ ]:
import os, time, random
from selenium import webdriver
from selenium.webdriver.common.by import By

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from urllib.parse import urlencode

from dotenv import load_dotenv
load_dotenv()

import warnings
warnings.filterwarnings("ignore")


from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
# from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver import ActionChains

In [2]:
driver = webdriver.Chrome()
driver.get('https://www.linkedin.com/login')
driver.title

'LinkedIn Login, Sign in | LinkedIn'

In [3]:
email = driver.find_element(By.ID, 'username')
email.send_keys(os.environ['EMAIL'])

password = driver.find_element(By.ID, 'password')
password.send_keys(os.environ['PASSWORD'])

password.submit()

In [27]:
## MAKE SURE TO USE ONLY THIS URL TO AVOID BEING STUCK IN ERRORS

url = "https://www.linkedin.com/company/stealth-startup-community/people/?facetGeoRegion=101282230"
driver.get(url)

In [ ]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time

def visit_and_connect(driver, profile_url, timeout=5):
    driver.get(profile_url)
    time.sleep(2)

    wait = WebDriverWait(driver, timeout)

    # Try direct Connect
    try:
        connect_btn = wait.until(
            EC.element_to_be_clickable(
                (By.XPATH, "//button[.//span[normalize-space()='Connect']]")
            )
        )
        print(f"connect_btn found: {connect_btn}")
        connect_btn.click()
        print("connect_clicked")
        return
    except TimeoutException:
        pass

    # Try Follow
    try:
        follow_btn = wait.until(
            EC.element_to_be_clickable(
                (By.XPATH, "//button[.//span[normalize-space()='Follow']]")
            )
        )
        follow_btn.click()
        print("follow_clicked")
        return
    except TimeoutException:
        pass

    # Try More → Connect
    try:
        more_btn = wait.until(
            EC.element_to_be_clickable(
                (By.XPATH, "//button[contains(@aria-label,'More actions')]")
            )
        )
        more_btn.click()
        time.sleep(1)

        dropdown_connect = wait.until(
            EC.element_to_be_clickable(
                (By.XPATH, "//div[@role='button']//span[normalize-space()='Connect']")
            )
        )
        dropdown_connect.click()
        print("connect_clicked_dropdown")
        return
    except TimeoutException:
        pass

    return "no_action"


In [28]:
while True:
    # collect profiles currently loaded
    profile_links = driver.find_elements(
        By.XPATH,
        "//a[contains(@href, 'linkedin.com/in/')]"
    )

    for link in profile_links:
        link = link.get_attribute("href")
        print("Visiting Profile:", link)
        visit_and_connect(driver, link)
        input("Press Enter to continue...")

    # try clicking "Show more results"
    try:
        show_more = driver.find_element(
            By.XPATH,
            "//button[.//span[text()='Show more results']]"
        )
        driver.execute_script("arguments[0].click();", show_more)
        time.sleep(2)
    except NoSuchElementException:
        break


Visiting Profile: https://www.linkedin.com/in/claudia-borrell-cores?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAADeJx8wB_pxaXXtGs8SQJ7bvH1xfQoODpHs


StaleElementReferenceException: Message: stale element reference: stale element not found
  (Session info: chrome=143.0.7499.170); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#staleelementreferenceexception
Stacktrace:
0   chromedriver                        0x0000000104bf4b2c cxxbridge1$str$ptr + 3033596
1   chromedriver                        0x0000000104becb30 cxxbridge1$str$ptr + 3000832
2   chromedriver                        0x00000001046e6b0c _RNvCsgXDX2mvAJAg_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 74196
3   chromedriver                        0x00000001046f745c _RNvCsgXDX2mvAJAg_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 142116
4   chromedriver                        0x00000001046f6558 _RNvCsgXDX2mvAJAg_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 138272
5   chromedriver                        0x00000001046ed148 _RNvCsgXDX2mvAJAg_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 100